##### Copyright 2020 Los autores de TensorFlow.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Normalizaciones

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/addons/tutorials/layers_normalizations"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/addons/blob/master/docs/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/addons/blob/master/docs/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/addons/docs/tutorials/layers_normalizations.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar libreta</a></td>
</table>


## Visión general

Este cuaderno brinda una breve introducción a las [capas de normalización](https://github.com/tensorflow/addons/blob/master/tensorflow_addons/layers/normalizations.py) de TensorFlow. Las capas admitidas actualmente son:

- **Normalización de grupos** (complementos de TensorFlow)
- **Normalización de instancias** (complementos de TensorFlow)
- **Normalización de capas** (TensorFlow Core)

La idea básica detrás de estas capas es normalizar la salida de una capa de activación para mejorar la convergencia durante el entrenamiento. A diferencia de la normalización por [lotes](https://keras.io/layers/normalization/) , estas normalizaciones no funcionan en lotes, sino que normalizan las activaciones de una sola muestra, lo que las hace adecuadas también para redes neuronales recurrentes.

Por lo general, la normalización se realiza calculando la media y la desviación estándar de un subgrupo en su tensor de entrada. También es posible aplicar una escala y un factor de compensación a esto.

$y_{i} = \frac{\gamma ( x_{i} - \mu )}{\sigma }+ \beta$

$ y$ : Salida

$x$ : Entrada

$\gamma$ : Factor de escala

$\mu$: media

$\sigma$: desviación estándar

$\beta$: factor de compensación

La siguiente imagen demuestra la diferencia entre estas técnicas. Cada subparcela muestra un tensor de entrada, con N como el eje del lote, C como el eje del canal y (H, W) como los ejes espaciales (alto y ancho de una imagen, por ejemplo). Los píxeles en azul están normalizados por la misma media y varianza, calculados agregando los valores de estos píxeles.

![](https://github.com/shaohua0116/Group-Normalization-Tensorflow/raw/master/figure/gn.png)

Fuente: (https://arxiv.org/pdf/1803.08494.pdf)

Los pesos gamma y beta se pueden entrenar en todas las capas de normalización para compensar la posible pérdida de capacidad de representación. Puede activar estos factores configurando el `center` o la bandera de la `scale` en `True` . Por supuesto, puede usar `initializers` , `constraints` y `regularizer` para `beta` y `gamma` para ajustar estos valores durante el proceso de entrenamiento. 

## Configuración

# Set up

### Instalar Tensorflow 2.0 y Tensorflow-Addons

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

### Preparación del conjunto de datos

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Tutorial de normalización de grupos

### Introducción

La normalización de grupo (GN) divide los canales de sus entradas en subgrupos más pequeños y normaliza estos valores en función de su media y varianza. Dado que GN trabaja en un solo ejemplo, esta técnica es independiente del tamaño del lote.

GN puntuó experimentalmente cerca de la normalización por lotes en tareas de clasificación de imágenes. Puede ser beneficioso usar GN en lugar de la Normalización por lotes en caso de que su tamaño de lote general sea bajo, lo que conduciría a un mal rendimiento de la normalización por lotes.

### Ejemplo

Dividir 10 canales después de una capa Conv2D en 5 subgrupos en una configuración estándar de "canales últimos":

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # Groupnorm Layer
  tfa.layers.GroupNormalization(groups=5, axis=3),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Tutorial de normalización de instancias

### Introducción

La normalización de instancias es un caso especial de normalización de grupos en el que el tamaño del grupo es del mismo tamaño que el tamaño del canal (o el tamaño del eje).

Los resultados experimentales muestran que la normalización de instancias funciona bien en la transferencia de estilo cuando se reemplaza la normalización por lotes. Recientemente, la normalización de instancias también se ha utilizado como reemplazo de la normalización por lotes en las GAN.

###Ejemplo Aplicar InstanceNormalization después de una capa Conv2D y usar una escala inicializada uniformada y un factor de compensación.

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tfa.layers.InstanceNormalization(axis=3, 
                                   center=True, 
                                   scale=True,
                                   beta_initializer="random_uniform",
                                   gamma_initializer="random_uniform"),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Tutorial de normalización de capas

### Introducción

La normalización de capas es un caso especial de normalización de grupo donde el tamaño del grupo es 1. La media y la desviación estándar se calculan a partir de todas las activaciones de una sola muestra.

Los resultados experimentales muestran que la normalización de capas es adecuada para las redes neuronales recurrentes, ya que funciona de forma independiente en el tamaño del lote.

### Ejemplo

Aplicar la normalización de capa después de una capa Conv2D y usar un factor de escala y desplazamiento. 

In [ ]:
model = tf.keras.models.Sequential([
  # Reshape into "channels last" setup.
  tf.keras.layers.Reshape((28,28,1), input_shape=(28,28)),
  tf.keras.layers.Conv2D(filters=10, kernel_size=(3,3),data_format="channels_last"),
  # LayerNorm Layer
  tf.keras.layers.LayerNormalization(axis=3 , center=True , scale=True),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_test, y_test)

## Literatura

[Norma de capa](https://arxiv.org/pdf/1607.06450.pdf)

[Norma de instancia](https://arxiv.org/pdf/1607.08022.pdf)

[Norma de grupo](https://arxiv.org/pdf/1803.08494.pdf)